# Azure open AI Service



## set up the env
* install the required packages
* create .env file,
  ```
  OPENAI_API_KEY="xxx"
  ```
* upload the file to /content folder
  * need to upload everytime when start a new session


In [ ]:
# work around to wrap the text from response
from IPython.display import HTML, display

# autowrap the output
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
import json

proxifier_log_fie = "data/summary/logs/Proxifier.txt"

SUMMARY = 'summary'
LOG = 'log'
def get_record_type(key):
  if key.startswith("#summary:#"):
    return SUMMARY
  else:
    return LOG

def load_data(file):
  data_log = {}
  data_summary = []
  current_key = None
  lines = []
  record_type = LOG

  with open(file, 'r') as file:
    for line in file:
      line = line.strip()
      if not line:
        continue

      if line.startswith("#"):
        if current_key is not None:
          if get_record_type(current_key) == SUMMARY:
            data_summary.append(lines)
          else:
            data_log[current_key.replace("#", "")] = lines

        current_key = line
        lines = []
      else:
        lines.append(line)

    if current_key is not None:
      data_summary.append(lines)
  return data_log, data_summary

p_data, p_summary = load_data(proxifier_log_fie)
print(len(p_data))
print( ', '.join(p_data.keys()))
print('\n'.join(p_data['1']))
print(p_summary[0])
print(len(p_data['2']))
print(len(p_summary))

100
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100
- proxy.cse.cuhk.edu.hk : 5070 open through proxy proxy.cse.cuhk.edu.hk : 5070 HTTPS
- proxy.cse.cuhk.edu.hk : 5070 open through proxy proxy.cse.cuhk.edu.hk : 5070 HTTPS
- proxy.cse.cuhk.edu.hk : 5070 open through proxy proxy.cse.cuhk.edu.hk : 5070 HTTPS
- proxy.cse.cuhk.edu.hk : 5070 open through proxy proxy.cse.cuhk.edu.hk : 5070 HTTPS
- proxy.cse.cuhk.edu.hk : 5070 open through proxy proxy.cse.cuhk.edu.hk : 5070 HTTPS
- proxy.cse.cuhk.edu.hk : 5070 close , 403 bytes sent , 426 bytes received , lifetime <1 sec
- proxy.cse.cuhk.edu.hk : 5070 open through proxy proxy.cse.cuhk.edu.hk : 5070 HTTPS
- 

In [ ]:
print('Proxifier sample log:\n')
print('\n'.join(p_data['2']))
print(f'\nProxifier sample summary:\n {p_summary[1][0]}')

print('\n\nZookeeper sample log:\n')
print('\n'.join(z_data['2']))
print(f'\nZookeeper sample summary:\n {z_summary[1][0]}')

Proxifier sample log:

- tcpconn.tencent.com : 80 open through proxy proxy.cse.cuhk.edu.hk : 5070 HTTPS
- tcpconn.tencent.com : 443 open through proxy proxy.cse.cuhk.edu.hk : 5070 HTTPS
- tcpconn.tencent.com : 80 close , 0 bytes sent , 0 bytes received , lifetime <1 sec
- tcpconn.tencent.com : 443 close , 0 bytes sent , 0 bytes received , lifetime <1 sec
- tcpconn5.tencent.com : 80 open through proxy proxy.cse.cuhk.edu.hk : 5070 HTTPS
- tcpconn5.tencent.com : 80 close , 0 bytes sent , 0 bytes received , lifetime <1 sec
- tcpconn5.tencent.com : 443 open through proxy proxy.cse.cuhk.edu.hk : 5070 HTTPS
- tcpconn5.tencent.com : 443 close , 0 bytes sent , 0 bytes received , lifetime <1 sec
- 183.60.48.250 : 443 open through proxy proxy.cse.cuhk.edu.hk : 5070 HTTPS
- proxy.cse.cuhk.edu.hk : 5070 open through proxy proxy.cse.cuhk.edu.hk : 5070 HTTPS
- proxy.cse.cuhk.edu.hk : 5070 close , 639 bytes sent , 339 bytes received , lifetime <1 sec
- proxy.cse.cuhk.edu.hk : 5070 open through proxy p

In [ ]:
import os
from openai import AzureOpenAI
import dotenv
import pandas as pd
import time
dotenv.load_dotenv('./.env')

client = AzureOpenAI(
  azure_endpoint = "https://aiops23.openai.azure.com/",
  api_key=os.getenv("OPENAI_API_KEY"),
  api_version="2023-09-15-preview"
)

def open_ai_summary(model, input):
  text = '\n'.join(input)

  template = f"""
  sure, I can create a template for summary as the following:
  opening port [Port Number], transmitting variable data ('[Transmitted Data]'), receiving ('[Received Data]') with average [Average Speed] MB/s, variance [Variance Speed] MB/s, finally closing the port."
  """

  response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "You are a networking specialist and understand how proxy works"},
        {"role": "user", "content": "Help me to create a concise summary less than 20 words, the output is a josn summary with the following keys: summary, throughput, anormly"},
        {"role": "assistant", "content": template},
        {"role": "user", "content":  text},
    ],
    temperature=0.3,
    max_tokens=250,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
  )

  print(response.choices[0].message.content)

  return response.choices[0].message.content

In [ ]:
engines = ['gpt3', 'gpt4']

results = pd.DataFrame(columns=['ref', 'gen3', 'gen4', 'log'])

for index, ref in enumerate(p_summary):
  # print(f"Key: {index}, Value: {input}")
  if index > 10:
    continue
  new_row = [ref]
  input = p_data[str(index+1)]
  for engine in engines:
    summary = open_ai_summary(engine, input)
    print(summary)
    new_row.append(summary)
    time.sleep(10)

  new_row.append('\n'.join(input))
  new_row = pd.DataFrame([new_row], columns=results.columns)
  results = pd.concat([results, new_row], ignore_index=True)

results.to_csv('openai_proxifier.csv', index=False)

NameError: ignored

In [ ]:
zookeeper_log_fie = "/content/code-and-datasets/data/summary/logs/Zookeeper.txt"
z_data, z_summary = load_data(zookeeper_log_fie)
print(len(z_data))
print( ', '.join(z_data.keys()))
print('\n'.join(z_data['1']))
print(z_summary[0])
print(len(z_data['2']))
print(len(z_summary))


100
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100
WARN Send worker leaving thread
INFO Received connection request /10.10.34.11 : 45328
WARN Connection broken for id 188978561024 , my id = 1 , error =
WARN Interrupting SendWorker
WARN Interrupted while waiting for message on queue
WARN Send worker leaving thread
INFO Received connection request /10.10.34.11 : 45329
WARN Connection broken for id 188978561024 , my id = 1 , error =
WARN Interrupting SendWorker
WARN Interrupted while waiting for message on queue
WARN Send worker leaving thread
INFO Received connection request /10.10.34.11 : 45336
INFO Received connection request /10.10.34.11 : 45338
W

In [ ]:

def open_ai_zk_summary(model, input, train):
  text = '\n'.join(input)
  # print(input)

  prompts=[]
  prompts.append({"role": "system", "content": "You are a zookeeper specialist"})
  prompts.append({"role": "user", "content": "Help me to create a concise summary less than 20 words, \
  the output is a josn summary with the following keys: summary, anormly"})
  prompts.append({"role": "assistant", "content": "yes, I can do it."})

  logs, summary = train[0], train[1]

  for i in range(len(train[0])):
    userContent = '\n'.join(logs[i])
    assistantContent = summary[i][0]
    prompts.append({"role": "user", "content": f"""create a summary for: {userContent}"""})
    # data = f""""summary": {assistantContent}"""
    prompts.append({"role": "assistant", "content": assistantContent})

  prompts.append({"role": "user", "content": f"create a summary for: {text}"})

  # for i in range(len(prompts)):
  #   print(prompts[i])

  response = client.chat.completions.create(
    model=model,
    messages=prompts,
    temperature=0.3,
    max_tokens=250,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
  )

  print(f'response:{response.choices[0].message.content}')

  return response.choices[0].message.content

In [ ]:

#  openai migrate

engines = ['gpt3', 'gpt4']

results = pd.DataFrame(columns=['ref', 'gen3', 'gen4', 'log'])

# use first 10 rows as training set
trains = ([],[])
for i in range(5):
  trains[0].append(z_data[str(index+1)])
  trains[1].append(z_summary[i][0])

print(len(trains[0][0]))
print(len(trains[1][0]))

for index, ref in enumerate(z_summary):
  # print(f"Key: {index}, Value: {input}")
  if index < 10 or index > 20:
    continue
  new_row = [ref]
  input = z_data[str(index+1)]
  for engine in engines:
    summary = open_ai_zk_summary(engine, input, trains)
    # print(summary)
    new_row.append(summary)
    time.sleep(10)

  new_row.append('\n'.join(input))
  new_row = pd.DataFrame([new_row], columns=results.columns)
  results = pd.concat([results, new_row], ignore_index=True)

results.to_csv('openai_zookeeper.csv', index=False)

20
113
response:{
  "summary": "Processed session terminations and expirations, accepted socket connection, caught end of stream exception, and closed socket connection.",
  "anomaly": "None"
}


KeyboardInterrupt: ignored